In [1]:
import Pkg;
Pkg.activate(@__DIR__);
Pkg.instantiate()

  Activating environment at `~/tinympc-julia/state_constraints/Project.toml`


In [2]:
using RobotZoo:Quadrotor
using RobotDynamics
using ForwardDiff
using TrajOptPlots
using BlockDiagonals
using LinearAlgebra
using StaticArrays
using SparseArrays

using MeshCat
using ColorTypes
using GeometryBasics: HyperRectangle, Cylinder, Vec, Point, Mesh
using CoordinateTransformations
using Rotations

using Plots
using Printf


include("../quaternion-stuff.jl")

E (generic function with 1 method)

In [3]:
# Quadrotor parameters
m = 0.035
ℓ = 0.046/sqrt(2)
J = [16.6e-6 0.83e-6 0.72e-6;
    0.83e-6 16.6e-6 1.8e-6;
    0.72e-6 1.8e-6 29.3e-6];
gravity = 9.81
thrustToTorque = 0.0008
scale = 65535
kt = 2.245365e-6*scale # u is PWM in range [0...1]
km = kt*thrustToTorque #4.4733e-8

# From "design of a trajectory tracking controller for a nanoquadcopter"
# J = [1.395e-5 0 0;
#     0 1.436e-5 0;
#     0 0 2.173e-5];
# kt = .2025
# km = .11

# h = 1/8000
# h = 1/5000
# h = 1/1000
# h = 1/500
# h = 1/250
# h = 1/100
h = 1/50
# h = 1/25
# h = 1/10

Nx = 13     # number of states (quaternion)
Nx̃ = 12     # number of states (linearized)
Nu = 4      # number of controls
nx = Nx̃
nu = Nu

#Goal state
# uhover = (m*gravity/kt/4)*ones(4) # m = 30g and max thrust = 60g
uhover = [.5; .5; .5; .5]
# uhover = [0.701, 0.701, 0.832, 0.732]
rg = [0.0; 0; 0.0]
qg = [1.0; 0; 0; 0]
vg = zeros(3)
ωg = zeros(3)
xg = [rg; qg; vg; ωg];

In [4]:
function quad_dynamics(x,u)
    r = x[1:3]
    q = x[4:7]/norm(x[4:7]) #normalize q just to be careful
    v = x[8:10]
    ω = x[11:13]
    Q = qtoQ(q)
    
    ṙ = v
    q̇ = 0.5*L(q)*H*ω
    
    b = 1/m
  
    v̇ = [0; 0; -gravity] + (1/m)*Q*[zeros(2,4); kt*ones(1,4)]*u 
  
    ω̇ = J\(-hat(ω)*J*ω + [-ℓ*kt -ℓ*kt ℓ*kt ℓ*kt; -ℓ*kt ℓ*kt ℓ*kt -ℓ*kt; -km km -km km]*u)
    
    return [ṙ; q̇; v̇; ω̇]
  end
  function quad_dynamics_rk4(x,u)
    #RK4 integration with zero-order hold on u
    f1 = quad_dynamics(x, u)
    f2 = quad_dynamics(x + 0.5*h*f1, u)
    f3 = quad_dynamics(x + 0.5*h*f2, u)
    f4 = quad_dynamics(x + h*f3, u)
    xn = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
    xn[4:7] = xn[4:7]/norm(xn[4:7]) #re-normalize quaternion
    return xn
  end

quad_dynamics_rk4 (generic function with 1 method)

In [5]:
vis = Visualizer()
quad_model = Quadrotor()
TrajOptPlots.set_mesh!(vis, quad_model)

function TransformPlane(visObject, center, dir)

    tol = 1e-4
    x_axis = [1.0; 0.0; 0.0]

    # Compute axis angle
    dot = x_axis'*dir
    if dot > 1-tol
        axis = x_axis
    elseif dot < -1+tol
        axis = -x_axis
    else
        axis = cross(x_axis, dir)
    end
    angle = acos(x_axis'*dir)

    settransform!(visObject, Translation(center...) ∘ LinearMap(AngleAxis(angle, axis...)))
end

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /home/tinympc/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


TransformPlane (generic function with 1 method)

In [6]:
# Create reference sketch trajectory
Tfinal = 5
N = Int(Tfinal/h)+1

X̃ref = [zeros(nx) for i = 1:N]
# Xref = [[0; 0; 1; 1; zeros(9)] for i = 1:N] # Point
Xref = [[0; t; 1; 1; zeros(9)] for t in range(-1.5, 1.5, length=N)] # Line
# Xref = [[sin(t); cos(t); 1; 1; zeros(9)] for t in range(-pi/2, 3*pi/2, length=N)] # Circle
# Xref = [[sin(2*t); 2*cos(t); 1; 1; zeros(9)] for t in range(-pi/2, 3*pi/2, length=N)] # Figure-eight
Uref = [zeros(nu) for i = 1:N-1]

# Compute reference velocity from reference position
for i = 1:N-1
    Xref[i][8:10] = (Xref[i+1][1:3] - Xref[i][1:3])/h
end

# Convert (13 state) Xref to reduced form (12 state) X̃ref
for k = 1:N
    x = Xref[k]
    qx = x[4:7]
    ϕ = qtorp(L(qg)'*qx)   
    X̃ref[k] = [x[1:3]; ϕ; x[8:10]; x[11:13]]
end

# Set initial state
x0 = X̃ref[1] + [0; 0; 0; zeros(9)]
# x0 = [0.1; 0.1; 0; zeros(9)]

# Visualize reference
delete!(vis["XrefLine"])
XrefLine = [Point(x_[1], x_[2], x_[3]) for x_ in Xref]
setobject!(vis["XrefLine"], Object(PointCloud(XrefLine), 
        LineBasicMaterial(color=Colors.RGBA(0.,0.45,1.)), "Line"))
;

In [7]:
# # Initialize normal distribution to add noise to the estimate of x
# using Random, Distributions
# Random.seed!(123)

In [8]:
using Libdl
# lib = dlopen("/Users/anoushkaalavill/Documents/REx_Lab/TinyMPC/lib_test") # library pointer
# func = dlsym(lib, "my_add") # function pointer
# function call_c(x, y)
#     @ccall $func(x::Cint, y::Cint)::Cint
# end
# output = call_c(5, 5)
# @show output

lib = dlopen("/home/tinympc/tinympc-crazyflie-firmware/examples/controller_tinympc_eigen_task/TinyMPC/build/src/tinympc/libtinympc.so") # library pointer
func_admm = dlsym(lib, "julia_sim_wrapper_solve_admm") # function pointer
function call_c_admm(x, u, mpc_iter, x_max, A_ineq)
    # @ccall $func(x::Ref{Cfloat}, y::Ref{Cfloat})::Cvoid # C takes 32-bit floats
    @ccall $func_admm(x::Ref{Cfloat}, u::Ref{Cfloat}, mpc_iter::Int, x_max::Ref{Cfloat}, A_ineq::Ref{Cfloat})::Cvoid # C takes 32-bit floats
end

func_lqr = dlsym(lib, "julia_sim_wrapper_solve_lqr") # function pointer
function call_c_lqr(x, u)
    # @ccall $func(x::Ref{Cfloat}, y::Ref{Cfloat})::Cvoid # C takes 32-bit floats
    @ccall $func_lqr(x::Ref{Cfloat}, u::Ref{Cfloat})::Cvoid # C takes 32-bit floats
end

call_c_lqr (generic function with 1 method)

In [9]:
include("../tinyMPC-ADMM-dt-state.jl")

speye(N) = spdiagm(ones(N))

# MPC setup
Nh = 15

# Create obstacle in MeshCat
obs_center = [0.0; 0.0; 0.8]
r_vis = 0.1
r_obstacle = 0.5

# Visualize sphere
bounding_material = MeshPhongMaterial(color=RGBA(1, 1, 1, 0.25))
setobject!(vis["obstacle"], HyperSphere(Point(obs_center...), r_vis))
setobject!(vis["bounding obstacle"], HyperSphere(Point(obs_center...), r_obstacle), bounding_material)

# Instantiate TinyMPC-ADMM variables
x = zeros(Float32, nx, Nh)
u = zeros(Float32, nu, Nh-1)

# State constraints
xmax = zeros(Float32, Nh)
Aineq = zeros(Float32, 3, Nh)

# Simulate
Nsim = N - Nh
xhist = [zeros(nx+1) for _ = 1:Nsim] # nx+1 because not reduced form
xhist[1] = [x0[1:3]; rptoq(x0[4:6]); x0[7:12]]
uhist = [zeros(nu) for _ = 1:Nsim-1]

prev_Δu = 0
for k = 1:Nsim-1 # Nsim-1 knotpoints for control, Nsim knotpoints for state

    # d = Normal()
    # noise = 0.01*rand(d, 12)
    x[:,1] = [xhist[k][1:3]; qtorp(xhist[k][4:7]); xhist[k][8:end]]# + noise # add noise to estimate of x

    # Solve
    call_c_admm(x, u, k, xmax, Aineq)

    # display(x)
    
    # Visualize solution x
    delete!(vis["xLinearized"])
    x_linearized = [Point(x[:,i][1], x[:,i][2], x[:,i][3]) for i in 1:Nh]
    setobject!(vis["xLinearized"], Object(PointCloud(x_linearized), 
    LineBasicMaterial(color=Colors.RGBA(1,0.6,0.0)), "Line"))

    # Visualize solution x projected onto sphere
    delete!(vis["xProjected"])
    x_projected = [Point(Aineq[:,i][1], Aineq[:,i][2], Aineq[:,i][3]) for i in 1:Nh]
    setobject!(vis["xProjected"], Object(PointCloud(x_projected), 
    LineBasicMaterial(color=Colors.RGBA(0.6,0.3,0.9)), "Line"))

    Δu = u
    # Roll out x_k+1 with actual dynamics
    # if prev_Δu == 0
    #     prev_Δu = Δu[:,1]
    # end
    uhist[k] = u[:,1] + uhover

    xhist[k+1] = quad_dynamics_rk4(xhist[k], uhist[k])
end

delete!(vis["xHistline"])
xHistline = [Point(x_[1], x_[2], x_[3]) for x_ in xhist]
setobject!(vis["xHistline"], Object(PointCloud(xHistline), 
LineBasicMaterial(color=Colors.RGBA(0.75,0.5,0.8)), "Line"))
     

MeshCat Visualizer with path /meshcat/xHistline at http://127.0.0.1:8700

In [13]:
X1 = [SVector{13}(x_) for x_ in xhist];
visualize!(vis, quad_model, Tfinal, X1)